In [2]:
import numpy as np
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import json
from datetime import datetime

In [ ]:
# Load the API key from the .env file
load_dotenv()
api_key = os.getenv("API_KEY")

# Fetch the odds data
sport_key = "basketball_ncaab"
regions = "eu"  # Regions: us, us2, uk, eu, au
markets = "h2h"  # Betting markets
odds_format = "decimal"  # Odds formats: decimal, american

url = f"https://api.the-odds-api.com/v4/sports/{sport_key}/odds"
params = {
    "apiKey": api_key,
    "regions": regions,
    "markets": markets,
    "oddsFormat": odds_format
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    
    # Add timestamp and parameters to the data
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    for game in data:
        game["timestamp"] = timestamp
        game["params"] = params
    
    # Generate a unique filename using a timestamp
    filename = f"odds_data_{datetime.now().strftime("%Y%m%d%H%M%S")}_{sport_key}_{regions}_{markets}_{odds_format}.json"
    
    # Save the JSON response to a file
    with open("json_requests/"+filename, "w") as json_file:
        json.dump(data, json_file, indent=4)
    
else:
    print(f"Error: {response.status_code} - {response.text}")

In [ ]:
# Prepare list for flattened data entries
flattened_data = []
for game in data:
    base_info = {
        "id": game["id"],
        "sport_key": game["sport_key"],
        "sport_title": game["sport_title"],
        "commence_time": game["commence_time"],
        "home_team": game["home_team"],
        "away_team": game["away_team"],
        "timestamp": game["timestamp"]
    }
    
    # Process each bookmaker and its markets
    for bookmaker in game["bookmakers"]:
        bookmaker_key = bookmaker["key"]
        for market in bookmaker["markets"]:
            market_key = market["key"]
            
            entry = base_info.copy()
            for outcome in market["outcomes"]:
                team = "home" if outcome["name"] == game["home_team"] else "away"
                # Generalized column name based on market type and team
                column_name = f"{bookmaker_key}_{team}_{market_key}_odds"
                entry[column_name] = outcome["price"]
            
            # Add the entry to flattened_data
            flattened_data.append(entry)

# Convert the flattened data into a DataFrame
df = pd.DataFrame(flattened_data).drop_duplicates()

# Collapse rows for each fixture by using `groupby` and `agg`
df = df.groupby(
    ["id", "sport_key", "sport_title", "commence_time", "home_team", "away_team", "timestamp"],
    as_index=False
).agg("first")  # Use "first" to take the first non-null value in each group

df

,id,sport_key,sport_title,commence_time,home_team,away_team,timestamp,livescorebet_eu_home_h2h_odds,livescorebet_eu_away_h2h_odds,marathonbet_home_h2h_odds,marathonbet_away_h2h_odds,nordicbet_away_h2h_odds,nordicbet_home_h2h_odds,gtbets_away_h2h_odds,gtbets_home_h2h_odds,betfair_ex_eu_away_h2h_odds,betfair_ex_eu_home_h2h_odds,betfair_ex_eu_away_h2h_lay_odds,betfair_ex_eu_home_h2h_lay_odds
0,00c0f2c81c89999de6a42005ccda02f7,basketball_ncaab,NCAAB,2024-11-13T03:00:00Z,Saint Mary's Gaels,Akron Zips,2024-11-12 11:48:47,1.08,8.00,1.09,7.10,7.05,1.08,NaN,NaN,NaN,NaN,NaN,NaN
1,016161d74bfde9c4edfe4abb16e2bde0,basketball_ncaab,NCAAB,2024-11-13T00:00:00Z,Temple Owls,Drexel Dragons,2024-11-12 11:48:47,1.33,3.40,1.32,3.30,3.30,1.31,3.40,1.33,NaN,NaN,NaN,NaN
2,048dfb6ef536af34dd49c667b4183acc,basketball_ncaab,NCAAB,2024-11-13T01:00:00Z,Illinois St Redbirds,Ohio Bobcats,2024-11-12 11:48:47,2.20,1.68,2.28,1.61,1.64,2.21,NaN,NaN,NaN,NaN,NaN,NaN
3,0f19bf0bbfabcce60debfd09b50b0c3c,basketball_ncaab,NCAAB,2024-11-13T03:00:00Z,Stanford Cardinal,Northern Arizona Lumberjacks,2024-11-12 11:48:47,1.03,12.50,1.03,11.25,10.50,1.03,10.05,1.07,NaN,NaN,NaN,NaN
4,21dcf14c2052953833a75d298f451f08,basketball_ncaab,NCAAB,2024-11-13T00:00:00Z,Indiana St Sycamores,SIU-Edwardsville Cougars,2024-11-12 11:48:47,1.26,3.95,NaN,NaN,4.15,1.21,4.20,1.24,NaN,NaN,NaN,NaN
5,2d67eff11bcb46032345f6ce45bfded2,basketball_ncaab,NCAAB,2024-11-13T00:00:00Z,New Hampshire Wildcats,Fairfield Stags,2024-11-12 11:48:47,2.02,1.81,1.91,1.86,1.77,2.01,1.83,2.00,NaN,NaN,NaN,NaN
6,3b53cf7895f50bf04422999b55ea9c81,basketball_ncaab,NCAAB,2024-11-12T23:30:00Z,La Salle Explorers,Cornell Big Red,2024-11-12 11:48:47,1.50,2.60,1.53,2.46,2.55,1.49,2.70,1.50,NaN,NaN,NaN,NaN
7,3fc06bf4223e2e7e00c5a7dcf6824189,basketball_ncaab,NCAAB,2024-11-13T01:00:00Z,Baylor Bears,Sam Houston St Bearkats,2024-11-12 11:48:47,1.07,9.00,1.08,7.60,9.35,1.04,9.75,1.07,NaN,NaN,NaN,NaN
8,4372cb7f774dc1bc1461d98c161eb408,basketball_ncaab,NCAAB,2024-11-13T00:00:00Z,Fordham Rams,Binghamton Bearcats,2024-11-12 11:48:47,1.09,7.50,1.13,5.65,7.10,1.08,7.05,1.12,NaN,NaN,NaN,NaN
9,4b4d05a034f5a008423b10403f697da1,basketball_ncaab,NCAAB,2024-11-13T02:00:12Z,Duke Blue Devils,Kentucky Wildcats,2024-11-12 11:48:47,1.40,3.00,1.39,2.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
pd.read_parquet("/Users/zacharias/Dropbox/Python/Project Jupiter/Project Zoccer/ncaa_basketball/project_ncaa_live/ncaa_team_scores_working_file.parquet").columns

Index(['date', 'Abilene Christian_M', 'Academy of Art_F', 'Academy of Art_M',
       'Adams State_F', 'Adams State_M', 'Adelphi_M', 'Adrian_M',
       'Agnes Scott_F', 'Air Force_F',
       ...
       'Yale_F', 'Yale_M', 'Yellowstone Christian_M', 'York_F', 'York_M',
       'Young Harris_F', 'Young Harris_M', 'Youngstown State_F',
       'Youngstown State_M', '_F'],
      dtype='object', length=2245)